# Creating Trajectory and Graph Objects, Store Local

In this Notebook we learn how to save our data as pickle files locally. This will help to create later on a CSV files with all the attributes we will need for prediction techniques. Remember we are saving objects so their size is considerable.

### Import Files & Modules

In [1]:
from Tonnetz_Select import fromMidiToPCS as fmpc
from Tonnetz_Select import analysisFromCorpus
from TrajectoryCalculationsWithClass import NewTrajectory, TrajectoryLookBefore
from graph_creation import CreateGraph
from FirstNotePosition import PlaceFirstNote
from structural_functions import mergeDicts
from Compliance_Function import complianceFunction
import os
import pickle
from itertools import islice, groupby
from operator import itemgetter
import music21 as ms
import pretty_midi as ptm


Use pickle to Save Files with ".p" extension

In [2]:
def pickleSave(dictOfGraphs, Composer_Name) :
    completeName = "Comparison_Results/GraphObjects_new/"+ Composer_Name + ".p"
    with open(completeName, 'wb') as config_dictionary_file:
      # Step 3
      pickle.dump(dictOfGraphs, config_dictionary_file)

Functions to find the Midi attributes :

In [3]:
def findInstruments(file):
    instruments = []
    time_signatures = []
    s = ms.converter.parse(file)
    for element in s.recurse() :
        if 'TimeSignature' in element.classes :
            time_signatures.append(element.ratioString)
        if 'Instrument' in element.classes :
            x = str(element)
            if x != '' :
                if ': ' in x :
                    newstring = x.split(': ', -1)[1]
                    instruments.append(newstring)
                else :
                    instruments.append(x)
    time_signatures = list(map(itemgetter(0), groupby(time_signatures)))
    return instruments, time_signatures

In [4]:
def TempoAdd(directory):
    s = ptm.PrettyMIDI(directory)
    tempo = s.estimate_tempo()
    return int(tempo)

The following Definitions are defines in python files but we added some details (found in AutoHarmonic Analysis)

In [5]:
def GraphOfNewPiece(newPiece, directory):
    '''Build a graph Object from a midi file.'''
    # extract the directory and the file extension from the piece
    shortfilename = os.path.splitext(os.path.basename(newPiece))[0]
    # Choose from where to parse (directory, corpus)
    if directory == 'corpus':
        file = ms.corpus.parse(newPiece)
#         chordList, Tonnetz, connectivity = analysisFromCorpus(file)
        # apply full compliance function for corpus
            # finds chordList and 2 Tonnetze
        chordList, (Tonnetz1, Tonnetz2) = complianceFunction(file, "Corpus")
    # add the instruments
        instruments, time_signatures = findInstruments(newPiece)
        tempo = []
    else:
        if newPiece.endswith(".mid") or newPiece.endswith(".MID"):
            complete_name = directory + '/' + newPiece
#             chordList, Tonnetz, connectivity = fmpc(complete_name)
            # apply full compliance function for File
            chordList, (Tonnetz1, Tonnetz2) = complianceFunction(complete_name, "File")
        # add the instruments
            instruments, time_signatures = findInstruments(complete_name)
            tempo = TempoAdd(complete_name)
#   build 2 different trajectories on 2 different Tonnetze
    # First Tonnetz 2 Graph
    firstPoint1 = PlaceFirstNote(chordList, Tonnetz1)
    trajectory1 = NewTrajectory(chordList, Tonnetz1, firstPoint1)
    trajectory1.addType('Trajectory Tonnetz 1')
    trajectory1.addInstruments(instruments)
    trajectory1.addTime_signatures(time_signatures)
    trajectory1.addTempo(tempo)
    graph1 = CreateGraph(trajectory1)
    graph1.addName(shortfilename)
    
    # Second Tonnetz 2 Graph
    firstPoint2 = PlaceFirstNote(chordList, Tonnetz2)
    trajectory2 = NewTrajectory(chordList, Tonnetz2, firstPoint2)
    trajectory2.addType('Trajectory Tonnetz 2')
    trajectory2.addInstruments(instruments)
    trajectory2.addTime_signatures(time_signatures)
    trajectory2.addTempo(tempo)
    graph2 = CreateGraph(trajectory2)
    graph2.addName(shortfilename)
    
    return graph1, graph2

In [6]:
def GetWorksByComposer(composerName, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    listofWorks = ms.corpus.getComposer(composerName)
    if len(listofWorks) > 60 :
        listofWorks = list(islice(listofWorks, 60))
    if len(listofWorks) > 0:
        for piece in listofWorks:
            try:
                print("Building Trajectory for ", piece)
                graph1, graph2 = GraphOfNewPiece(piece, 'corpus')
#                 addingLabels(composer, style, harmony, graph)
                graph1.addStyle(style)
                graph1.addComposer(composer)
                graph2.addStyle(style)
                graph2.addComposer(composer)
                listOfGraphs.append([graph1, graph2])
            except BaseException:
                print("--> Cannot build Trajectory for ", piece)
    return listOfGraphs

In [8]:
def GetWorksByDirectory(directory, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    for file in os.listdir(directory):
        if file.endswith(".mid") or file.endswith(
                ".MID") or file.endswith(".mxl") or file.endswith(".xml"):
            try:
                print("Building Trajectory for ", file)
                graph1, graph2 = GraphOfNewPiece(file, directory)
#                 addingLabels(composer, style, harmony, graph)
                graph1.addStyle(style)
                graph1.addComposer(composer)
                graph2.addStyle(style)
                graph2.addComposer(composer)
                listOfGraphs.append([graph1, graph2])
            except BaseException:
                print("--> Cannot build Trajectory for ", file)
    return listOfGraphs

### Begin Process (define Directory, composer and style)

In [9]:
mozart_quartets = GetWorksByDirectory('Midi_files/mozart_string_quartets', 'mozart', 'classical')

Building Trajectory for  k080-01.mid
1034
Building Trajectory for  k080-02.mid
1146
Building Trajectory for  k080-03.mid
470
Building Trajectory for  k080-04.mid
438
Building Trajectory for  k155-01.mid
933
Building Trajectory for  k155-02.mid
690
Building Trajectory for  k155-03.mid
474
Building Trajectory for  k156-01.mid
1182
Building Trajectory for  k156-02.mid
694
Building Trajectory for  k156-03.mid
--> Cannot build Trajectory for  k156-03.mid
Building Trajectory for  k157-01.mid
1909
Building Trajectory for  k157-02.mid
1183
Building Trajectory for  k157-03.mid
501
Building Trajectory for  k158-01.mid
1260
Building Trajectory for  k158-03.mid
588
Building Trajectory for  k160-02.mid
600
Building Trajectory for  k160-03.mid
1226
Building Trajectory for  k168-01.mid
656
Building Trajectory for  k168-02.mid
198
Building Trajectory for  k168-03.mid
150
Building Trajectory for  k169-01.mid
678
Building Trajectory for  k169-03.mid
211
Building Trajectory for  k169-04.mid
357
Building 

In [10]:
haydn_quartets = GetWorksByDirectory('Midi_files/haydn_string_quartets', 'haydn', 'classical')

Building Trajectory for  op01n0-01.mid
322
Building Trajectory for  op01n0-02.mid
269
Building Trajectory for  op01n0-03.mid
488
Building Trajectory for  op01n0-05.mid
405
Building Trajectory for  op01n1-01.mid
295
Building Trajectory for  op01n1-02.mid
187
Building Trajectory for  op01n1-03.mid
249
Building Trajectory for  op01n1-04.mid
132
Building Trajectory for  op01n1-05.mid
353
Building Trajectory for  op01n2-01.mid
487
Building Trajectory for  op01n2-02.mid
201
Building Trajectory for  op01n2-03.mid
335
Building Trajectory for  op01n2-04.mid
201
Building Trajectory for  op01n2-05.mid
243
Building Trajectory for  op01n3-01.mid
548
Building Trajectory for  op01n3-03.mid
234
Building Trajectory for  op01n3-04.mid
220
Building Trajectory for  op01n3-05.mid
411
Building Trajectory for  op01n4-01.mid
558
Building Trajectory for  op01n4-02.mid
260
Building Trajectory for  op01n4-03.mid
556
Building Trajectory for  op01n4-04.mid
226
Building Trajectory for  op01n4-05.mid
436
Building Tr

In [11]:
pickleSave(mozart_quartets, 'mozart_quartets')
pickleSave(haydn_quartets, 'haydn_quartets')

In [12]:
bach = GetWorksByComposer('bach', 'bach', 'barok')

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv1.6.mxl
124
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv10.7.mxl
61
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv101.7.mxl
61
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv102.7.mxl
71
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv103.6.mxl
84
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv104.6.mxl
67
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv108.6.mxl
91
Building Trajectory for  C:\Users\melki\AppData\Lo

In [13]:
beethoven = GetWorksByDirectory('Midi_files/Classical/Beethoven', 'beethoven', 'classical')
beethovenCorpus = GetWorksByComposer('beethoven', 'beethoven', 'classical')

Building Trajectory for  Cnpia523.mid
3793
Building Trajectory for  Concvi1.mid
5604
[7, 4] dict_items([(7, (104, 103))]) (104, 103) 3 2 1
--> Cannot build Trajectory for  Concvi1.mid
Building Trajectory for  Concvi23.mid
3140
Building Trajectory for  Coriolan.mid
1890
Building Trajectory for  Leonore1.mid
2331
Building Trajectory for  Leonore2.mid
3357
Building Trajectory for  Leonore3.mid
3696
[11, 6, 3] dict_items([(11, (103, 104)), (6, (104, 105))]) (103, 104) 4 3 2
--> Cannot build Trajectory for  Leonore3.mid
Building Trajectory for  String Quartet n1 op18 1mov.mid
2612
Building Trajectory for  String Quartet n1 op18 2mov.mid
1238
Building Trajectory for  String Quartet n1 op18 3mov.mid
1518
Building Trajectory for  String Quartet n1 op18 4mov.mid
2640
Building Trajectory for  String Quartet n16 op135 1mov.mid
1499
[1, 3, 5] dict_items([(1, (102, 102)), (5, (103, 102))]) (102, 102) 4 3 2
--> Cannot build Trajectory for  String Quartet n16 op135 1mov.mid
Building Trajectory for  S

7344
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\beethoven\opus74.mxl
4833


In [14]:
pickleSave(bach, 'bach')
pickleSave(beethoven, 'beethoven')
pickleSave(beethovenCorpus, 'beethovenCorpus')

In [15]:
chopin = GetWorksByDirectory('Midi_files/Classical/Chopin', 'chopin', 'classical')
chopinCorpus = GetWorksByComposer('chopin', 'chopin', 'classical')

Building Trajectory for  56336a_polonaise_op_53_(nc)smythe.mid
2674
Building Trajectory for  69521a_etude_(aeolian_harp)_op_25_no_1_(nc)smythe.mid
1308
Building Trajectory for  andante_polacca_22_(c)finley.mid
5719
Building Trajectory for  chopin_mazurkas_67_2.mid
591
Building Trajectory for  chopin_polonaise-fantasie_61_(c)lubetsky.mid
3987
Building Trajectory for  chopin_raindrop_prelude.mid
413
Building Trajectory for  chpn-p1.mid
246
Building Trajectory for  chpn-p10.mid
185
Building Trajectory for  chpn-p11.mid
147
Building Trajectory for  chpn-p12.mid
394
Building Trajectory for  chpn-p13.mid
384
Building Trajectory for  chpn-p14.mid
217
--> Cannot build Trajectory for  chpn-p14.mid
Building Trajectory for  chpn-p15.mid
375
Building Trajectory for  chpn-p16.mid
707
Building Trajectory for  chpn-p17.mid
348
Building Trajectory for  chpn-p18.mid
243
Building Trajectory for  chpn-p19.mid
544
Building Trajectory for  chpn-p2.mid
159
Building Trajectory for  chpn-p20.mid
59
Building T

C:\Users\melki\AppData\Local\Programs\Python\Python36-32\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Building Trajectory for  nocturne_15_1_(c)unknown.mid
1008
Building Trajectory for  nocturne_62_2_(c)unknown.mid
984
Building Trajectory for  nocturne_72_1_(c)unknown.mid
748
Building Trajectory for  nocturne_9_1_(c)unknown.mid
1204
Building Trajectory for  nocturne_9_2_(c)unknown.mid
622
Building Trajectory for  nocturne_9_3_(c)unknown.mid
1749
Building Trajectory for  nocturne_c_minor_(c)inoue.mid
538
Building Trajectory for  nocturne_op_9_1_2979_r_(nc)smythe.mid
1615
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\chopin\mazurka06-2.krn
285


In [16]:
mozart = GetWorksByDirectory('Midi_files/Classical/Mozart', 'mozart', 'classical')
mozartCorpus = GetWorksByComposer('mozart', 'mozart', 'classical')

Building Trajectory for  Ad_fug.mid
1814
[7, 8] dict_items([(7, (103, 105))]) (103, 105) 3 2 1
--> Cannot build Trajectory for  Ad_fug.mid
Building Trajectory for  Conc_clarinet-1.mid
4176
Building Trajectory for  Conc_clarinet-2.mid
679
Building Trajectory for  Conc_clarinet-3.mid
2994
Building Trajectory for  mz_333_1.mid
4110
Building Trajectory for  mz_333_2.mid
1434
Building Trajectory for  mz_333_3.mid
2193
Building Trajectory for  mz_570_1.mid
2670
Building Trajectory for  mz_570_2.mid
1020
Building Trajectory for  mz_570_3.mid
1234
Building Trajectory for  piano_sonata_310_1_(c)oguri.mid
2321
Building Trajectory for  piano_sonata_310_2_(c)oguri.mid
2418
Building Trajectory for  piano_sonata_310_3_(c)oguri.mid
1430
Building Trajectory for  piano_sonata_330_1_(c)oguri.mid
2038
Building Trajectory for  piano_sonata_330_2_(c)oguri.mid
790
Building Trajectory for  piano_sonata_330_3_(c)oguri.mid
2006
Building Trajectory for  piano_sonata_331_1_(c)oguri.mid
2601
[4, 5] dict_items([(4

In [17]:
schumann = GetWorksByDirectory('Midi_files/Classical/Schumann', 'schumann', 'classical')
schumannCorpus = GetWorksByComposer('schumann', 'schumann', 'classical')

Building Trajectory for  By-The-Fireside-Opus-15-(8th-Movement).mid
363
Building Trajectory for  Catch-Me!-Opus-15-(3rd-Movement).mid
297
Building Trajectory for  Child-Falling-Asleep-Opus-15-(12th-Movement).mid
367
Building Trajectory for  Curious-Story-Opus-15-(2nd-Movement).mid
370
Building Trajectory for  Fantasy-Pieces-in-A-Flat-Opus-111-Nr-2.mid
1179
Building Trajectory for  Fantasy-Pieces-in-C-Minor-Opus-111-Nr-1.mid
1445
Building Trajectory for  Fantasy-Pieces-in-C-Minor-Opus-111-Nr-3.mid
918
Building Trajectory for  Fantasy-Pieces-Opus-12-Nr-6.mid
756
Building Trajectory for  Foreign-Land-And-People-Opus-15-(1st-Movement).mid
431
Building Trajectory for  Friendly-Landscape-From-Waldscenen.mid
469
Building Trajectory for  Frightening-Opus-15-(11th-Movement).mid
562
Building Trajectory for  Fruhlingsnacht.mid
1041
Building Trajectory for  Important-Event-Opus-15-(6st-Movement).mid
216
Building Trajectory for  In-The-Evening-Opus-12-Nr-1.mid
1115
Building Trajectory for  In-The-N

In [18]:
palestrina = GetWorksByComposer('palestrina', 'palestrina', 'renaissance')
monteverdi = GetWorksByComposer('monteverdi', 'monteverdi', 'renaissance')

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus.krn
198
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_00.krn
208
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_01.krn
180
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_02.krn
267
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_03.krn
149
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_04.krn
169
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_05.krn
208
Bu

205
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_I_37.krn
214
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_I_38.krn
181
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_I_39.krn
205
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.1.mxl
405
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.1.rntxt
228
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.10.mxl
190
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\mo

133
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.15.mxl
262
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.15.rntxt
66
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.16.mxl
248
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.16.rntxt
169
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.17.mxl
140
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.17.rntxt
88
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\musi

In [19]:
jazz = GetWorksByDirectory('Midi_files/Standard_Jazz/Random', 'Unknown', 'jazz')

Building Trajectory for  A-Getting-Sentimental-Over-You-1.mid
516
Building Trajectory for  A-Taste-Of-Honey.mid
863
Building Trajectory for  After-You-Gone.mid
1547
Building Trajectory for  All-Of-Me-2.mid
2630
Building Trajectory for  All-The-Things-You-Are.mid
2922
Building Trajectory for  All-The-Way.mid
1383
Building Trajectory for  Alone-Together.mid
874
Building Trajectory for  Antropology.mid
1680
Building Trajectory for  Anything-Goes.mid
1172
Building Trajectory for  As-Time-Goes-By.mid
1091
Building Trajectory for  Autumn-Leaves.mid
1647
Building Trajectory for  Basin-Street-Blues-1.mid
969
Building Trajectory for  Basin-Street-Blues-2.mid
1285
Building Trajectory for  Bewitched-1.mid
970
Building Trajectory for  Big-Band-Medley-(Medley).mid
1334
Building Trajectory for  Billie's-Bounce-1.mid
1831
[10, 0] dict_items([(10, (104, 102))]) (104, 102) 3 2 1
--> Cannot build Trajectory for  Billie's-Bounce-1.mid
Building Trajectory for  Billie's-Bounce-2.mid
1829
[7, 0, 10] dict_it

### Storing

In [20]:
pickleSave(chopin, 'chopin')
pickleSave(chopinCorpus, 'chopinCorpus')

pickleSave(schumann, 'schumann')
pickleSave(schumannCorpus, 'schumannCorpus')

pickleSave(mozart, 'mozart')
pickleSave(mozartCorpus, 'mozartCorpus')

pickleSave(palestrina, 'palestrina')

pickleSave(monteverdi, 'monteverdi')

pickleSave(jazz, 'jazz')